In [ ]:
import astropy.coordinates as coord
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline

from scipy.stats import binned_statistic, scoreatpercentile

First generate a bunch of uniform positions within a sphere of radius 200 pc:

In [ ]:
def get_isotropic_angles(size=1):
    phi = np.random.uniform(0, 360, size=N) * u.deg
    theta = np.arccos(2*np.random.uniform(size=N) - 1) * u.rad
    return {'phi': phi, 'theta': theta}

In [ ]:
def get_stats(dx, v_mag, size, n_bins=16):
    # First make a bunch of isotropic, uniform comoving pairs within a sphere
    r = 200*u.pc * np.cbrt(np.random.uniform(size=N))
    pos1 = coord.PhysicsSphericalRepresentation(r=r, **get_isotropic_angles(size=N))

    r = dx * np.ones(N)
    pos2 = pos1 + coord.PhysicsSphericalRepresentation(r=r, **get_isotropic_angles(size=N)) 

    v_basis = coord.PhysicsSphericalRepresentation(r=np.ones(N), **get_isotropic_angles(size=N)) 
    v_basis = v_basis.represent_as(coord.CartesianRepresentation).xyz
    v = coord.CartesianDifferential(v_mag * v_basis)

    icrs1 = coord.ICRS(pos1.represent_as(coord.CartesianRepresentation)
                           .with_differentials(v))
    icrs2 = coord.ICRS(pos2.represent_as(coord.CartesianRepresentation)
                           .with_differentials(v))
    
    # compute ∆µ and ∆vtan
    dpm = np.sqrt((icrs1.pm_ra_cosdec-icrs2.pm_ra_cosdec)**2 + (icrs1.pm_dec - icrs2.pm_dec)**2)
    dvtan = (dpm * icrs1.distance).to(u.km/u.s, u.dimensionless_angles())
    
    # compute binned stats
    bins = np.linspace(0, 200+(200/n_bins/2), n_bins)
    
    res = dict(dpm=[], dvtan=[])
    res['dpm'].append(binned_statistic(icrs1.distance.to(u.pc).value, 
                                       dpm.to(u.mas/u.yr).value, 
                                       bins=bins, statistic=np.median))
    res['dpm'].append(binned_statistic(icrs1.distance.to(u.pc).value, 
                                       dpm.to(u.mas/u.yr).value, 
                                       bins=bins, statistic=lambda x: scoreatpercentile(x, 5)))
    res['dpm'].append(binned_statistic(icrs1.distance.to(u.pc).value, 
                                       dpm.to(u.mas/u.yr).value, 
                                       bins=bins, statistic=lambda x: scoreatpercentile(x, 95)))
    
    res['dvtan'].append(binned_statistic(icrs1.distance.to(u.pc).value, 
                                         dvtan.value, 
                                         bins=bins, statistic=np.median))
    res['dvtan'].append(binned_statistic(icrs1.distance.to(u.pc).value, 
                                         dvtan.value, 
                                         bins=bins, statistic=lambda x: scoreatpercentile(x, 5)))
    res['dvtan'].append(binned_statistic(icrs1.distance.to(u.pc).value, 
                                         dvtan.value, 
                                         bins=bins, statistic=lambda x: scoreatpercentile(x, 95)))
    
    res['x'] = 0.5 * (bins[1:] + bins[:-1])
    
    return res

In [ ]:
colors = ['tab:blue', 'tab:orange', 'tab:green']

In [ ]:
fig,axes = plt.subplots(2, 1, figsize=(6,8), sharex=True)

for i,v in enumerate([100, 30, 5]*u.km/u.s):
    stats = get_stats(dx=1*u.pc, v_mag=v, size=1E6, n_bins=21)
    for j,name in enumerate(['dpm', 'dvtan']):
        axes[j].plot(stats['x'], stats[name][0].statistic, 
                     label='$v-v_\odot = {0.value:.0f}$ {0.unit:latex_inline}'.format(v),
                     marker='', linestyle='-', linewidth=2, color=colors[i], zorder=10)
        axes[j].fill_between(stats['x'], stats[name][1].statistic, stats[name][2].statistic,
                             color=colors[i], alpha=0.5, zorder=1, linewidth=0)
    
axes[0].set_yscale('log')
axes[0].set_ylim(1E-2, 1E2)
axes[0].set_yticks(10. ** np.arange(-2, 2+1, 1))
axes[0].set_yticklabels([0.01, 0.1, 1, 10, 100])
axes[0].set_ylabel(r'$|\Delta\bar{{\mu}}|$ [{0:latex_inline}]'.format(u.mas/u.yr))
axes[0].legend(loc='upper right', fontsize=14)
axes[0].set_title(r'$|\Delta\bar{x}| = 1$ [pc]', fontsize=22)

axes[1].set_yscale('log')
axes[1].set_ylim(1E-2, 5E1)
axes[1].set_xlim(0, 200)
axes[1].set_yticks(10. ** np.arange(-2, 1+1, 1))
axes[1].set_yticklabels([0.01, 0.1, 1, 10])
axes[1].set_ylabel(r'$|\Delta\bar{{v}}_{{\rm tan}}|$ [{0:latex_inline}]'.format(u.km/u.s))

axes[1].set_xlabel(r'distance, $d_\odot$ [{0}]'.format(u.pc))

fig.tight_layout()

In [ ]:
fig,axes = plt.subplots(2, 1, figsize=(6,8), sharex=True)

for i,dx in enumerate([10, 1, 0.1]*u.pc):
    stats = get_stats(dx=dx, v_mag=30*u.km/u.s, size=1E6, n_bins=21)
    for j,name in enumerate(['dpm', 'dvtan']):
        label = r'$|\Delta\bar{{x}}| = {0.value:.0f}$ {0.unit:latex_inline}'.format(dx)
        if i == 2:
            label = r'$|\Delta\bar{{x}}| = {0.value:.1f}$ {0.unit:latex_inline}'.format(dx)
        axes[j].plot(stats['x'], stats[name][0].statistic, label=label,
                     marker='', linestyle='-', linewidth=2, color=colors[i], zorder=10)
        axes[j].fill_between(stats['x'], stats[name][1].statistic, stats[name][2].statistic,
                             color=colors[i], alpha=0.5, zorder=1, linewidth=0)
    
axes[0].set_yscale('log')
axes[0].set_ylim(5E-2, 5E2)
axes[0].set_yticks(10. ** np.arange(-2, 2+1, 1))
axes[0].set_yticklabels([0.01, 0.1, 1, 10, 100])
axes[0].set_ylabel(r'$|\Delta\bar{{\mu}}|$ [{0:latex_inline}]'.format(u.mas/u.yr))
axes[0].legend(loc='upper right', fontsize=14)
axes[0].set_title(r'$v-v_\odot = 30$ [{0:latex_inline}]'.format(u.km/u.s), fontsize=22)

axes[1].set_yscale('log')
axes[1].set_ylim(1E-2, 5E1)
axes[1].set_xlim(0, 200)
axes[1].set_yticks(10. ** np.arange(-2, 1+1, 1))
axes[1].set_yticklabels([0.01, 0.1, 1, 10])
axes[1].set_ylabel(r'$|\Delta\bar{{v}}_{{\rm tan}}|$ [{0:latex_inline}]'.format(u.km/u.s))

axes[1].set_xlabel(r'distance, $d_\odot$ [{0}]'.format(u.pc))

fig.tight_layout()